Import all necessary libraries and install everything you need for training:

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
!pip install -q transformers

In [3]:
# Install the simpletransformers
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [ ]:
# Install wandb
!pip install -q wandb

In [ ]:
import wandb

In [ ]:
# Login to wandb
wandb.login()

In [ ]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()


### Import the data

In [ ]:
# FTD
train_df = pd.read_csv("/kaggle/input/genredatasetscomparison/FTD-train.txt", sep="\t", index_col=0)
dev_df = pd.read_csv("/kaggle/input/genredatasetscomparison/FTD-dev.txt", sep = "\t", index_col = 0)
test_df = pd.read_csv("/kaggle/input/genredatasetscomparison/FTD-test.txt", sep = "\t", index_col = 0)

print("FTD train shape: {}, Dev shape: {}, Test shape: {}.".format(train_df.shape, dev_df.shape, test_df.shape))

In [ ]:
train_df.head()

## Training and testing

We will use the multilingual XLM-RoBERTa model
https://huggingface.co/xlm-roberta-base

In [ ]:
# Create a file to save results into (you can find it under Data: Output). Be careful, run this step only once to not overwrite the results file.
results = []

with open("FTD-Experiments-Results.json", "w") as results_file:
    json.dump(results,results_file, indent= "")

In [ ]:
# Open the main results file:

previous_results_file = open("./FTD-Experiments-Results.json")
previous_results = json.load(previous_results_file)
len(previous_results)

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()
LABELS

## Hyperparameter sweeps

In [ ]:
# Configure the WandB sweep for hyperparameter search
sweep_config = {
    "method": "grid",  # random, grid, bayes
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [10, 20, 30, 50, 70, 90]},
    },
}

In [ ]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="FTD-learning-hyperparameter-sweep")

In [ ]:
# Add logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# Create a function that will be ran inside the sweep
def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "labels_list": LABELS,
            "no_cache": True,
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            "evaluate_during_training":True,
            'logging_steps': 10,
            'evaluate_during_training_steps': 10,
            "use_cached_eval_features": True,
            "reprocess_input_data": True,
            "silent": True,
            "wandb_project": 'FTD-learning-hyperparameter-sweep',
            "sweep_config":wandb.config
            }
        )

    # Train the model
    roberta_base_model.train_model(train_df, eval_df=dev_df)

    # Evaluate the model
    roberta_base_model.eval_model(dev_df)

    # Sync wandb
    wandb.join()

In [ ]:
# Run the sweeps
wandb.agent(sweep_id, train, count= 10)

## Training

I manually checked for the best parameters, experimenting only with the epochs by using evaluation during training. See the report in Wandb about it. The best epoch number revealed to be 10.

In [14]:
# Initialize Wandb
wandb.init(project="FTD-learning-manual-hyperparameter-search", entity="tajak", name="saving-trained-model")

In [15]:
# Import LOGGING - makes the Transformer logger less verbose
import logging

logging.basicConfig(level=logging.INFO)

# Get root logger (all other loggers will be derived from this logger's
# properties)
logger = logging.getLogger()
logger.warning("Is this working?") 

# Get the logger for the huggingface/transformers library.
transformers_logger = logging.getLogger("transformers")

# Set the logging level to warning, meaning display warnings and worse, but 
# don't display any `INFO` logs.
transformers_logger.setLevel(logging.WARNING)

In [16]:
# Calculate how many steps will each epoch have
# Num steps in epoch = training samples / batch size
steps_per_epoch = int(849/8)
steps_per_epoch

In [17]:
# I'll evaluate per every 10th epoch - per 1060 steps.

In [23]:
# Create a TransformerModel
roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": 10,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            "evaluate_during_training": True,
            "evaluate_during_training_steps": steps_per_epoch*10,
            "evaluate_during_training_verbose": True,
            "use_cached_eval_features": True,
            'reprocess_input_data': True,
            "labels_list": LABELS,
            "no_cache": True,
            # Disable no_save: True if you want to save the model
            #"no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            "wandb_project": 'FTD-learning-manual-hyperparameter-search',
            }
        )

In [22]:
# Train the model
roberta_base_model.train_model(train_df)

In [ ]:
# See where the model is saved - check if the same things are in outputs - maybe there is the final model (check model_args)
!ls /kaggle/working/outputs/best_model

In [ ]:
# Save the trained model to Wandb
trained_model_artifact = wandb.Artifact("FTD-classifier", type="model", description="a model trained on the FTD dataset")
trained_model_artifact.add_dir("/kaggle/working/outputs/best_model")
run.log_artifact(trained_model_artifact)

In [ ]:
# To load the model from Wandb
model_name = "FTD-classifier"
# Use the latest version of the model
model_at = run.use_artifact(model_name + ":lastest")
# Download the directory
model_dir = model.at_download()

# Loading a local save
model = ClassificationModel(
    "xlmroberta", model_dir)

In [ ]:
# Loading a local save
model = ClassificationModel(
    "xlmroberta", "/kaggle/working/outputs/best_model")

In [ ]:
def testing(test_df, test_name):
    """
    This function takes the test dataset and applies the trained model on it to infer predictions.
    It also prints and saves a confusion matrix, calculates the F1 scores and saves the results in a list of results.

    Args:
    - test_df (pandas DataFrame)
    - test_name
    """
    # Get the true labels
    y_true = test_df.labels
    
    model = roberta_base_model

    # Calculate the model's predictions on test
    def make_prediction(input_string):
        return model.predict([input_string])[0][0]

    y_pred = test_df.text.apply(make_prediction)

    # Calculate the scores
    macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
    micro = f1_score(y_true, y_pred, labels=LABELS,  average="micro")
    print(f"Macro f1: {macro:0.3}, Micro f1: {micro:0.3}")

    # Plot the confusion matrix:
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, '{:d}'.format(z), ha='center', va='center')
    classNames = LABELS
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)
    plt.title(f"{test_name}")

    plt.tight_layout()
    fig1 = plt.gcf()
    plt.show()
    plt.draw()
    fig1.savefig(f"Confusion-matrix-{test_name}.png",dpi=100)

    # Save the results:
    rezdict = {
        "experiment": test_name,
        "num_train_epochs": 10,
        "train_batch_size":8,
        "learning_rate": 1e-5,
        "microF1": micro,
        "macroF1": macro,
        "y_true": y_true.tolist(),
        "y_pred": y_pred.tolist(),
        }
    previous_results.append(rezdict)

    #Save intermediate results (just in case)
    backup = []
    backup.append(rezdict)
    with open(f"backup-results-{test_name}.json", "w") as backup_file:
        json.dump(backup,backup_file, indent= "")

In [ ]:
testing(dev_df, "hyperparameter-search-final-evaluation-10-epochs")

In [ ]:
testing(test_df, "FTD-test-10-epochs")

In [ ]:
previous_results = previous_results[:2]

In [ ]:
# Compare the results by creating a dataframe from the previous_results dictionary:
results_df = pd.DataFrame(previous_results)

results_df

In [ ]:
# Save the file with updated results.
with open("FTD-Experiments-Results.json", "w") as results_file:
    json.dump(previous_results,results_file, indent= "")

In [ ]:
!ls /kaggle/working/runs/Jul28_10-01-31_228df68315e6/events.out.tfevents.1659002491.228df68315e6.33.0

In [ ]:
# Save the model to wandb
# "model.h5" is saved in wandb.run.dir & will be uploaded at the end of training
roberta_base_model.save(os.path.join(wandb.run.dir, "model.h5"))

# Save a model file manually from the current directory
roberta_base_model.save('model.h5')

# Save all files that currently exist containing the substring "ckpt":
roberta_base_model.save('../logs/*ckpt*')

# Save any files starting with "checkpoint" as they're written to:
roberta_base_model.save(os.path.join(wandb.run.dir, "checkpoint*"))

In [ ]:
# Clean the GPU cache
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()